In [234]:
from importlib import reload
reload(metric)

<module 'metric' from 'D:\\device_usage\\metric.py'>

In [235]:
import pandas as pd
from schema import schema
import metric
from metric import *

In [159]:
# Read data
df = pd.read_csv(r'data.csv', dtype=schema, encoding='utf-8')
# Pre-processing
df.drop(['state'], axis=1, inplace=True)
df.fillna(0, inplace=True)

In [160]:
df['plan_c'].value_counts()

10.0      847459
0.0       188760
30.0       63016
6.0            5
2048.0         2
1024.0         2
Name: plan_c, dtype: int64

In [161]:
# Remove outlier  (plan_c == 6)
df = df[df['plan_c']!=6]

In [176]:
# Assign all $0 plan to $10
df_nozeroplan = df.copy()
df_nozeroplan['plan_c'] = df['plan_c'].replace(0.0, 10.0)

In [163]:
df['best_plan'] = df['used'].apply(get_best_plan)

In [164]:
df['best_plan'].value_counts()

2       451771
5       363756
10      272963
30        8477
100       1436
300        671
500        135
700         21
2048         5
1024         4
Name: best_plan, dtype: int64

In [165]:
df_nozeroplan.head()

,iccid,used,plan_c,left,ac_time,product,company,carrier,plan_s,month
0,898607B4071790000000,3.931,10.0,6.069,2018-01-16 15:21:11,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
1,898607B4071790000001,0.000,10.0,10.000,2018-01-16 15:21:44,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
2,898607B4071790000002,4.877,10.0,5.123,2018-01-16 15:19:09,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
3,898607B4071790000003,4.259,10.0,5.741,2018-02-09 11:10:04,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
4,898607B4071790000004,1.740,10.0,8.260,2018-01-16 15:21:28,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2


In [166]:
df_nozeroplan['plan_c'].value_counts()

10.0      1036219
30.0        63016
2048.0          2
1024.0          2
Name: plan_c, dtype: int64

Calculate best plan to subscribe to for each device, without considering data pool.

In [177]:
df_nozeroplan['best_plan'] = df_nozeroplan['used'].apply(get_best_plan)

In [178]:
df_nozeroplan.head()

,iccid,used,plan_c,left,ac_time,product,company,carrier,plan_s,month,best_plan
0,898607B4071790000000,3.931,10.0,6.069,2018-01-16 15:21:11,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,5
1,898607B4071790000001,0.000,10.0,10.000,2018-01-16 15:21:44,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,2
2,898607B4071790000002,4.877,10.0,5.123,2018-01-16 15:19:09,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,5
3,898607B4071790000003,4.259,10.0,5.741,2018-02-09 11:10:04,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,5
4,898607B4071790000004,1.740,10.0,8.260,2018-01-16 15:21:28,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,2


In [180]:
df_nozeroplan['best_plan'].value_counts()

2       451771
5       363756
10      272963
30        8477
100       1436
300        671
500        135
700         21
2048         5
1024         4
Name: best_plan, dtype: int64

In [ ]:
# Actual cost in previous months, using 0 MB for all plan_c == 0 devices
cost_original = dict()
for i in [2,3,4,5]:
    cost_original.update({i: calculate_cost_with_data_pool(df[df['month']==i])})
print(cost_original)

In [238]:
# Actual cost in previous months, using 10 MB for all plan_c == 0 devices
cost_nozeroplan = dict()
for i in [2,3,4,5]:
    best_cost = calculate_cost_with_data_pool(df_nozeroplan[df_nozeroplan['month']==i], verbose=True)
    cost_nozeroplan.update({i: best_cost})
print(cost_nozeroplan)  # beware, this cost assumes all plan_c==0 are using 10 MB plan

Total devices: 202974
Total cost: 631332.0
Total usage:803452.0540000113
   plan_c   count        used  pool_size  remaining_quota  pool_utilization  \
0    10.0  191769  722826.027  1917690.0      1194863.973          0.376925   
1    30.0   11205   80626.027   336150.0       255523.973          0.239851   

   plan_cost  extra_cost  total_cost  
0   575307.0           0    575307.0  
1    56025.0           0     56025.0  
Total devices: 207341
Total cost: 645285.0
Total usage:942648.4099999964
   plan_c   count        used  pool_size  remaining_quota  pool_utilization  \
0    10.0  195710  830002.003  1957100.0      1127097.997          0.424098   
1    30.0   11631  112646.407   348930.0       236283.593          0.322834   

   plan_cost  extra_cost  total_cost  
0   587130.0           0    587130.0  
1    58155.0           0     58155.0  
Total devices: 215835
Total cost: 674167.0
Total usage:981954.5119999921
   plan_c   count        used  pool_size  remaining_quota  pool_utiliza

In [239]:
# Best plan cost
cost_best_plan = dict()
for i in [2,3,4,5]:
    print("Month: {}".format(i))
    best_cost = calculate_cost_with_data_pool(df[df['month']==i], col_plan = 'best_plan', verbose=True)
    cost_best_plan.update({i: best_cost})
print(cost_best_plan)

Month: 2
Total devices: 202974
Total cost: 366133.0
Total usage:803452.053999995
   best_plan   count        used  pool_size  remaining_quota  \
0          2   52916   51794.632     105832        54037.368   
1          5  140749  640646.656     703745        63098.344   
2         10    7760   65716.938      77600        11883.062   
3         30    1435   31081.186      43050        11968.814   
4        100      84    5441.666       8400         2958.334   
5        300      20    4088.497       6000         1911.503   
6        500       6    2357.997       3000          642.003   
7        700       4    2324.482       2800          475.518   

   pool_utilization  plan_cost  extra_cost  total_cost  
0          0.489404    52916.0           0     52916.0  
1          0.910339   281498.0           0    281498.0  
2          0.846868    23280.0           0     23280.0  
3          0.721979     7175.0           0      7175.0  
4          0.647817      672.0           0       672.0  


# Strategy_1
## Rules
1. If a device was activated before this month, use last month's usage as prediction and evaluate this month's plan.
2. If a device was not activated before this month (i.e. first seen this month), use plan_s as this month's plan.

In [237]:
cost_strategy_1 = dict()
for mth in [3, 4, 5]:
    df_this_month = df_nozeroplan[df_nozeroplan['month']==mth]
    df_this_month.set_index('iccid', inplace=True)
    df_last_month = df_nozeroplan[df_nozeroplan['month']==mth-1][['iccid','best_plan']]
    df_last_month.rename({'best_plan': 'plan_st_1'}, axis=1, inplace=True)
    df_last_month.set_index('iccid', inplace=True)
    # Get last month's best plan
    df_this_month = df_this_month.join(df_last_month, how='left')
    # Fill plan_s == 0 with 10.0
    df_this_month['plan_s'].replace(0.0, 10.0, inplace=True)
    # Fill in NaN best_plan_lastmonth
    df_this_month['plan_st_1'].fillna(df_this_month['plan_s'], inplace=True)
    # Calculate cost of this month, using strategy 1
    print("Month {}:".format(mth))
    cost_strategy_1.update({mth: calculate_cost_with_data_pool(df_this_month, col_plan='plan_st_1', verbose=True)})
print(cost_strategy_1)

Month 3:
Total devices: 207341
Total cost: 395978.317631997
Total usage:942648.409999997
    plan_st_1   count        used  pool_size  remaining_quota  \
0         2.0   52916   89147.630   105832.0        16684.370   
1         5.0  140749  714625.193   703745.0       -10880.193   
2        10.0    9438   78976.325    94380.0        15403.675   
3        30.0    4121   44771.055   123630.0        78858.945   
4       100.0      84    5248.153     8400.0         3151.847   
5       300.0      20    4928.189     6000.0         1071.811   
6       500.0       6    2723.587     3000.0          276.413   
7       700.0       4    2208.745     2800.0          591.255   
8      2048.0       1      15.853     2048.0         2032.147   
9      3072.0       1       0.000     3072.0         3072.000   
10     4096.0       1       3.680     4096.0         4092.320   

    pool_utilization  plan_cost    extra_cost     total_cost  
0           0.842350    52916.0      0.000000   52916.000000  
1   

In [229]:
# Compare difference
lst_cost = [cost_nozeroplan, cost_strategy_1, cost_best_plan]
df_compare = pd.DataFrame(lst_cost, index=['Paid cost', 'Cost with stg 1', 'Cost with best plan'])

In [230]:
df_compare.head()

,2,3,4,5
Paid cost,631332.0,645285.000000,674167.0,726531.00000
Cost with stg 1,NaN,395978.317632,459697.0,539172.17952
Cost with best plan,366133.0,418069.000000,447816.0,438988.00000
